In [69]:
import numpy as np

# These params come from the D1_merged_all_cracks_6micron_griddata.vtk file in /data. They are 1 less than the 
#  DIMENSIONS specified because we are using the center of each voxel.

def get_voxel_centers_as_array(num_x_pts, num_y_pts, num_z_pts, increment, file):
    total_voxels = num_x_pts * num_y_pts * num_z_pts
    natee = 0
    next_print = total_voxels / 10
    
    voxel_centers = np.empty((num_x_pts, num_y_pts, num_z_pts), dtype=int)
    with open(file, 'r') as voxel_centers_file:
        voxel_centers_file.readline()
        for line in voxel_centers_file:
            tokens = line.split(',')
            x = int(float(tokens[0]))
            y = int(float(tokens[1]))
            z = int(float(tokens[2]))
            grain_id = int(tokens[3])
            
            # We divide the voxel_centers by the increment so that we fill every index of the list
            voxel_centers[x // increment][y // increment][z // increment] = grain_id
            
            # Print progress
            natee += 1
            if natee >= next_print:
                print('Progress: ', natee / total_voxels)
                next_print += total_voxels / 10
    return voxel_centers

In [70]:
# This will give the nearest voxel center (x,y,z) based on the voxel size being the length of 1 side of the voxel

def find_nearest_voxel_center(x, y, z, voxel_size):
    # This works because / gives a float while // gives an int. So this says that it's in one boundary until the next
    voxel_x = int(((x + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_y = int(((y + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_z = int(((z + (voxel_size / 2)) // voxel_size) * voxel_size)
    return voxel_x, voxel_y, voxel_z

In [71]:
# Reads in the crack_front_points file. dadN3Dline is the output we're concerned with.

def get_crack_surface_points(input_file):
    with open(input_file, 'r') as crack_points_file:
        crack_points_file.readline()
        crack_points = []
        for line in crack_points_file:
            tokens = line.split(',')
            dadN = tokens[0]
            crack_id = tokens[1]
            theta = tokens[2]
            x = tokens[3]
            y = tokens[4]
            z = tokens[5]
            on_crack_front = tokens[6].strip()
            crack_points.append((dadN, crack_id, theta, x, y, z, on_crack_front))
    return crack_points

In [72]:
# The 4 rightmost digits of the grain_id identify it. The other digits are to show which crack front it's on and 
#  whether it's above or below the crack front

def get_actual_grain_id(long_grain_id):
    return long_grain_id % 10000

In [73]:
from math import atan, pi

# Returns true if the given point point (x,y) is in front of the crack front for the given crack front.
# Remember that X and Y are flipped.

def is_in_front_of_crack_surface(in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, x, y, on_crack_front, actual_x = None, actual_y = None):
    
    # If we already know this x,y is in front
    if (x,y) in in_front_of_crack_surface:
        return True
    
    # if we already know this x,y isn't in front
    elif (x,y) in not_in_front_of_crack_surface:
        return False
    
    # We need to figure it out
    else:
        # This reference point is from the 0-crack_front_growth_rates_1500ppcf_transformed.csv file for CF_id=0
        reference_x = -1.9932
        reference_y = 363.26

        # Find the opposite and adjacent of a triangle formed between reference and given (x,y)
        opposite_point = abs(x - reference_x)
        adjacent_point = abs(y - reference_y)

        # If the line isn't exactly vertical (i.e.: does not have undefined slope)
        if adjacent_point != 0:
            theta_point = atan(opposite_point / adjacent_point)
            
            # If the given point (x,y) is to the right of the reference point, then we got the wrong theta and we need to flip it
            if y > reference_y:
                theta_point = pi - theta_point
                
            # Formula from the point of reference to the point being tested: x=ay+c
            if on_crack_front:
                a = (x - reference_x) / (y - reference_y) # Rise over Run. X and Y are switched in this data
                c = -a * y + x # Solve x=ay+c for c and stick a point in
            else:
                a = (actual_x - reference_x) / (actual_y - reference_y) # Rise over Run. X and Y are switched in this data
                c = -a * actual_y + actual_x # Solve x=ay+c for c and stick a point in

        # Otherwise, the line is vertical, so set theta to pi halves and say we don't know a and c
        else:
            theta_point = pi / 2
            a = None
            c = None

            
        if on_crack_front:
            # Scan the crack front until we find the two crack front points nearest theta-wise to the give (x,y)
            # TODO: This could be sped up with a binary search
            crack_index1 = 0
            crack_index2 = crack_index1 + 1
            num_pts = len(this_crack_front)
            theta_crack2 = 0

            while (crack_index2 + 1 < num_pts and theta_crack2 < theta_point):
                temp_crack_1x = float(this_crack_front[crack_index1][3])
                temp_crack_1y = float(this_crack_front[crack_index1][4])
                temp_crack_2x = float(this_crack_front[crack_index2][3])
                temp_crack_2y = float(this_crack_front[crack_index2][4])

                # Find the opposite and adjacent of a triangle formed between reference and given (x,y)
                opposite_crack1 = abs(temp_crack_1x - reference_x)
                adjacent_crack1 = abs(temp_crack_1y - reference_y)
                opposite_crack2 = abs(temp_crack_2x - reference_x)
                adjacent_crack2 = abs(temp_crack_2y - reference_y)

                # If the line isn't exactly vertical (i.e.: does not have undefined slope)
                if adjacent_crack1 != 0:
                    theta_crack1 = atan(opposite_crack1 / adjacent_crack1)
                    if temp_crack_1y > reference_y:
                        theta_crack1 = pi - theta_crack1
                else:
                    theta_crack1 = pi / 2

                if adjacent_crack2 != 0:
                    theta_crack2 = atan(opposite_crack2 / adjacent_crack2)
                    if temp_crack_2y > reference_y:
                        theta_crack2 = pi - theta_crack2
                else:
                    theta_crack2 = pi / 2

                crack_index1 += 1
                crack_index2 = crack_index1 + 1

            crack_1x = float(this_crack_front[crack_index1][3])
            crack_1y = float(this_crack_front[crack_index1][4])
            crack_2x = float(this_crack_front[crack_index2][3])
            crack_2y = float(this_crack_front[crack_index2][4])

            # If the line isn't vertical, find its slope and intercept
            if crack_1y != crack_2y:
                # Formula for points on the crack front: x = by + d
                b = (crack_1x - crack_2x) / (crack_1y - crack_2y) # Rise over Run. X and Y are switched in this data
                d = -b * crack_1y + crack_1x # Solve x = by + d for d and use the point
            else:
                b = None
                d = None

        else: # not on_crack_front
            if a != 0:
                b = -1 / a
                d = (1 / a) * actual_y + actual_x
            else:
                b = None
                d = None
            
        # If we found both lines
        if a is not None and b is not None:
            intersection_y = (d - c) / (a - b) # Remember X and Y are flipped
            intersection_x = a * intersection_y + c
        # Otherwise, we're missing a line
        else:
            # If we got the line from the reference to the point (x,y), but not the crack front line
            if a is not None:
                if on_crack_front:
                    intersection_y = crack_1y # We know the crack front line is vertical and y is always the same
                else:
                    intersection_y = y
                intersection_x = a * intersection_y + c

            # Otherwise, if we got the crack front line but not the point line
            elif b is not None:
                intersection_y = y # The point line is vertical, so its y is always the same
                intersection_x = b * intersection_y + d

            # This shouldn't happen cause they are parallel or coincident if it does...If it does I'll figure out how to deal with it
            else:
                raise RuntimeError('Both lines are vertical...')

        distance_point = ((y - reference_y)**2 + (x - reference_x)**2)**.5
        distance_intersection = ((intersection_y - reference_y)**2 + (intersection_x - reference_x)**2)**.5

        if on_crack_front:
            if distance_point >= distance_intersection:
                in_front_of_crack_surface[(x,y)] = True
                return True
            else:
                not_in_front_of_crack_surface[(x,y)] = False
                return False
        else:
            on_line = b * y + d # Find the x coordinate of the perpendicular line
            if x >= on_line: # If the point we're checking is above or on that line in the x direction
                return True
            else:
                return False

In [74]:
def get_distance(x1, y1, z1, x2, y2, z2):
    return ((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)**.5

In [75]:
# http://code.activestate.com/recipes/81188-binary-search/
def binary_search(seq, t):
    min = 0
    max = len(seq) - 1
    while True:
        if max < min:
            return min
        m = (min + max) // 2
        if seq[m][3] < t:
            min = m + 1
        elif seq[m][3] > t:
            max = m - 1
        else:
            return m

In [76]:
def insert_into_queue(queue, xyzd):
    index = binary_search(queue, xyzd[3])
    queue.insert(index, xyzd)

In [77]:
from collections import deque

# Given an array of the voxel centers arr[x][y][z] = grain_id at those 'coordinates / voxel_size'. Returns the nearest 
#  voxel center indices that have a different grain_id associated with them. Basically, just does a breadth-first search 
#  with some optimizations

def find_nearest_grain_boundary(voxel_centers, checked_voxels, x, y, z, grain_id, in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, on_crack_front, voxel_size = 2):
    grain_id = get_actual_grain_id(grain_id)
    queue = [(x,y,z,0)]
    original_x, original_y, original_z = get_xyz_from_voxel_centers_index(x, y, z, voxel_size)
    
    while len(queue) > 0:
        u = queue.pop(0)
        current_x, current_y, current_z, current_d = u # These are indices
        actual_x, actual_y, actual_z = get_xyz_from_voxel_centers_index(current_x, current_y, current_z, voxel_size) # 2 is voxel size...
        
        # voxel_centers[current_x][current_y][current_z] is the grain_id of the voxel we're checking
        current_grain_id = get_actual_grain_id(voxel_centers[current_x][current_y][current_z])
        
        if current_grain_id != grain_id: 
            return (current_x, current_y, current_z, current_grain_id)
        
        checked_voxels[current_x][current_y][current_z] = True
        
        if current_x < len(voxel_centers) - 1 and not checked_voxels[current_x + 1][current_y][current_z] \
                and is_in_front_of_crack_surface(in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, actual_x + voxel_size, actual_y, on_crack_front, original_x, original_y):
            distance = get_distance(x*voxel_size, y*voxel_size, z*voxel_size, actual_x + voxel_size, actual_y, actual_z)
            insert_into_queue(queue, (current_x + 1, current_y, current_z, distance))
            checked_voxels[current_x + 1][current_y][current_z] = True
            
        if current_x > 0 and not checked_voxels[current_x - 1][current_y][current_z] \
                and is_in_front_of_crack_surface(in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, actual_x - voxel_size, actual_y, on_crack_front, original_x, original_y):
            distance = get_distance(x*voxel_size, y*voxel_size, z*voxel_size, actual_x - voxel_size, actual_y, actual_z)
            insert_into_queue(queue, (current_x - 1, current_y, current_z, distance))
            checked_voxels[current_x - 1][current_y][current_z] = True
            
        if current_y < len(voxel_centers[0]) - 1 and not checked_voxels[current_x][current_y + 1][current_z] \
                and is_in_front_of_crack_surface(in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, actual_x, actual_y + voxel_size, on_crack_front, original_x, original_y):
            distance = get_distance(x*voxel_size, y*voxel_size, z*voxel_size, actual_x, actual_y + voxel_size, actual_z)
            insert_into_queue(queue, (current_x, current_y + 1, current_z, distance))
            checked_voxels[current_x][current_y + 1][current_z] = True
            
        if current_y > 0 and not checked_voxels[current_x][current_y - 1][current_z] \
                and is_in_front_of_crack_surface(in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, actual_x, actual_y - voxel_size, on_crack_front, original_x, original_y):
            distance = get_distance(x*voxel_size, y*voxel_size, z*voxel_size, actual_x, actual_y - voxel_size, actual_z)
            insert_into_queue(queue, (current_x, current_y - 1, current_z, distance))
            checked_voxels[current_x][current_y - 1][current_z] = True
            
        if current_z < len(voxel_centers[0][0]) - 1 and not checked_voxels[current_x][current_y][current_z + 1] \
                and is_in_front_of_crack_surface(in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, actual_x, actual_y, on_crack_front, original_x, original_y):
            distance = get_distance(x*voxel_size, y*voxel_size, z*voxel_size, actual_x, actual_y, actual_z + voxel_size)
            insert_into_queue(queue, (current_x, current_y, current_z + 1, distance))
            checked_voxels[current_x][current_y][current_z + 1] = True
            
        if current_z > 0 and not checked_voxels[current_x][current_y][current_z - 1] \
                and is_in_front_of_crack_surface(in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, actual_x, actual_y, on_crack_front, original_x, original_y):
            distance = get_distance(x*voxel_size, y*voxel_size, z*voxel_size, actual_x, actual_y, actual_z - voxel_size)
            insert_into_queue(queue, (current_x, current_y, current_z - 1, distance))
            checked_voxels[current_x][current_y][current_z - 1] = True
    
    raise RuntimeError('Grain boundary not found')

In [78]:
from collections import deque

# Given an array of the voxel centers arr[x][y][z] = grain_id at those 'coordinates / voxel_size'. Returns the nearest 
#  voxel center that has a different grain_id associated with it. Basically, just does a breadth-first search...
# Stuff is currently commented out because the data broke the commented out code on crack 4, it may be faster though

def find_nearest_voxel_in_front_of_crack(in_front_of_crack_surface, not_in_front_of_crack_surface, voxel_centers, checked_voxels, xi, yi, zi, this_crack_front, on_crack_front):
    # TODO: Convert to distance similar to above...
    queue = deque([(xi,yi,zi)])
    
    while len(queue) > 0:
        u = queue.popleft()
        current_x, current_y, current_z = u # These are indices
        actual_x, actual_y, actual_z = get_xyz_from_voxel_centers_index(current_x, current_y, current_z, 2) # 2 is voxel size...
        
        if is_in_front_of_crack_surface(in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, actual_x, actual_y, on_crack_front):
            return (actual_x, actual_y, actual_z)
        
        checked_voxels[current_x][current_y][current_z] = True
        if current_x < len(voxel_centers) - 1 and not checked_voxels[current_x + 1][current_y][current_z]:
            queue.append((current_x + 1, current_y, current_z))
            checked_voxels[current_x + 1][current_y][current_z] = True
            
        if current_x > 0 and not checked_voxels[current_x - 1][current_y][current_z]:
            queue.append((current_x - 1, current_y, current_z))
            checked_voxels[current_x - 1][current_y][current_z] = True
            
        if current_y < len(voxel_centers[0]) - 1 and not checked_voxels[current_x][current_y + 1][current_z]:
            queue.append((current_x, current_y + 1, current_z))
            checked_voxels[current_x][current_y + 1][current_z] = True
            
        if current_y > 0 and not checked_voxels[current_x][current_y - 1][current_z]:
            queue.append((current_x, current_y - 1, current_z))
            checked_voxels[current_x][current_y - 1][current_z] = True
    
    raise RuntimeError('Voxel center not found...')

In [79]:
# Get the index in voxel_centers from x,y,z

def get_voxel_centers_index(x, y, z, voxel_size):
    return x // voxel_size, y // voxel_size, z // voxel_size

In [80]:
# Get the x,y,z from voxel_centers indices

def get_xyz_from_voxel_centers_index(xi, yi, zi, voxel_size):
    return xi * voxel_size, yi * voxel_size, zi * voxel_size

In [58]:
# Time: ~ 2 minutes
# These params come from the D1_merged_all_cracks_6micron_griddata.vtk file in /data. They are 1 less than the 
#  DIMENSIONS specified because we are using the center of each voxel.

voxel_centers = get_voxel_centers_as_array(201, 376, 301, 2, '../../data/1-voxel-centers.csv')
print('done')

Progress:  0.10000001758367279
Progress:  0.20000003516734557
Progress:  0.3000000087918364
Progress:  0.4000000263755092
Progress:  0.5
Progress:  0.6000000175836728
Progress:  0.7000000351673455
Progress:  0.8000000087918364
Progress:  0.9000000263755091
done


In [59]:
# Gets the crack front points from file

crack_surface_points = get_crack_surface_points('../../data/0-point-surface.csv')

In [81]:
# Save the nearest grain boundaries for the points on each crack front

from datetime import datetime
print('Opening Time')
print(datetime.now().time())

# Setup
voxel_size = 2

# Loop through all the crack fronts
for crack_id_test in ['0','1','2','3','4','5','6','7','8']:
    print('crack_id_test: ', crack_id_test)
    
    previously_looked_up = {}
    next_print = 3.15 / 10

    # Set up crack_front points so we can check whether points are in front or behind later
    this_crack_front = []
    for crack_surface_point in crack_surface_points:
        this_crack_id = int(crack_surface_point[1])
        if this_crack_id == int(crack_id_test) and crack_surface_point[6] == 'True':
            this_crack_front.append(crack_surface_point)
            
    # This keeps track of lookups we've made to see if a voxel is in front of the current crack surface
    in_front_of_crack_surface = {}
    not_in_front_of_crack_surface = {}

    with open('../../data/2-nearest-grain-boundary.csv', 'a') as file:
        # Only write the header the first time
        if crack_id_test == '0':
            file.write('dadN,crack_id,theta,x,y,z,on_crack_front,grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,nearest_grain_boundary_z,nearest_grain_boundary_id\n')
        for crack_surface_point in crack_surface_points:
            dadN3Dline, crack_id, theta, x, y, z, on_crack_front = crack_surface_point
            if crack_id_test == crack_id:
                x = float(x)
                y = float(y)
                z = float(z)
                on_crack_front = on_crack_front == 'True'

                # Get the nearest voxel_center for the x,y,z of the crack_front
                nearest_voxel_xyz = find_nearest_voxel_center(x, y, z, voxel_size)
                nearest_voxel_x, nearest_voxel_y, nearest_voxel_z = nearest_voxel_xyz

                if nearest_voxel_x < 0 or nearest_voxel_y < 0 or nearest_voxel_z < 0: # Ignore the point if isn't near a real voxel
                    continue

                # If the nearest voxel is behind the crack front, spread out to find one that's in front
                if on_crack_front:
                    if not is_in_front_of_crack_surface(in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, nearest_voxel_x, nearest_voxel_y, on_crack_front, x, y):
                        checked_false_temp = np.full(voxel_centers.shape, False, np.bool)
                        voxel_centers_index_x_, voxel_centers_index_y_, voxel_centers_index_z_ = \
                        get_voxel_centers_index(nearest_voxel_x, nearest_voxel_y, nearest_voxel_z, voxel_size)
                        nearest_voxel_xyz = find_nearest_voxel_in_front_of_crack(in_front_of_crack_surface, not_in_front_of_crack_surface, voxel_centers, 
                                     checked_false_temp, voxel_centers_index_x_, voxel_centers_index_y_, voxel_centers_index_z_, this_crack_front, on_crack_front)
                        nearest_voxel_x, nearest_voxel_y, nearest_voxel_z = nearest_voxel_xyz


                # We need to keep track of what voxels we search
                checked_false = np.full(voxel_centers.shape, False, np.bool)

                # Get the grain_id of the voxel we're looking at
                voxel_centers_index_x, voxel_centers_index_y, voxel_centers_index_z = \
                    get_voxel_centers_index(nearest_voxel_x, nearest_voxel_y, nearest_voxel_z, voxel_size)
                nearest_grain_id = get_actual_grain_id(voxel_centers[voxel_centers_index_x][voxel_centers_index_y][voxel_centers_index_z])

                # Find the x,y,z,grain_id of the nearest grain boundary
                if nearest_voxel_xyz in previously_looked_up:
                    grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id = previously_looked_up[nearest_voxel_xyz]
                else:
#                     try:
                    grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id = find_nearest_grain_boundary(voxel_centers,\
                          checked_false, voxel_centers_index_x, voxel_centers_index_y, voxel_centers_index_z, nearest_grain_id, in_front_of_crack_surface, not_in_front_of_crack_surface, this_crack_front, on_crack_front)
                    previously_looked_up[nearest_voxel_xyz] = (grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id)

                # Got back to x,y,z from indices
                grain_boundary_x, grain_boundary_y, grain_boundary_z = get_xyz_from_voxel_centers_index(grain_boundary_index_x, \
                                                                grain_boundary_index_y, grain_boundary_index_z, voxel_size)

                temp = str(dadN3Dline)+','+str(crack_id)+','+str(theta)+','+str(x)+','+str(y)+','+str(z)+','+str(on_crack_front)+','+str(nearest_grain_id)+','+\
                    str(grain_boundary_x)+','+str(grain_boundary_y)+','+str(grain_boundary_z)+','+str(grain_boundary_id)+'\n'
                file.write(temp)

                if float(theta) > next_print:
                    print('Progress: ', float(theta) / 3.15)
                    next_print += 3.15 / 10

    print('Closing Time')
    print(datetime.now().time())
print('done')

Opening Time
23:30:04.419246
crack_id_test:  0
Progress:  0.10039682539682539
Progress:  0.20013015873015874
Progress:  0.30053015873015876
Progress:  0.40025396825396825
Progress:  0.5006666666666667
Progress:  0.6003809523809523
Progress:  0.7001269841269842
Progress:  0.8005396825396826
Progress:  0.9002539682539682
Closing Time
23:33:37.258053
crack_id_test:  1
Progress:  0.10039682539682539
Progress:  0.20013015873015874
Progress:  0.30053015873015876
Progress:  0.40025396825396825
Progress:  0.5006666666666667
Progress:  0.6003809523809523
Progress:  0.7001269841269842
Progress:  0.8005396825396826
Progress:  0.9002539682539682
Closing Time
23:35:19.286223
crack_id_test:  2
Progress:  0.10039682539682539
Progress:  0.20013015873015874
Progress:  0.30053015873015876
Progress:  0.40025396825396825
Progress:  0.5006666666666667
Progress:  0.6003809523809523
Progress:  0.7001269841269842
Progress:  0.8005396825396826
Progress:  0.9002539682539682
Closing Time
23:39:24.098076
crack_id

In [58]:
# in_front_of_crack_surface = {}
# not_in_front_of_crack_surface = {}
# checked_voxels = np.full(voxel_centers.shape, False, np.bool)
# xi = 0
# yi = 332 // 2
# zi = 304 // 2
# this_crack_front = []
# crack_id_test = '1'
# for crack_surface_point in crack_surface_points:
#     this_crack_id = int(crack_surface_point[1])
#     if this_crack_id == int(crack_id_test) and crack_surface_point[6] == 'True':
#         this_crack_front.append(crack_surface_point)
# on_crack_front = True
# find_nearest_voxel_in_front_of_crack(in_front_of_crack_surface, not_in_front_of_crack_surface, voxel_centers, checked_voxels, xi, yi, zi, this_crack_front, on_crack_front)

(0, 330, 304)

In [52]:
# len(in_front_of_crack_surface)

605

In [46]:
# x = 24
# y = 408.3
# on_crack_front = False
# actual_x = 25.34694999
# actual_y = 407.4925242

# from math import atan, pi

# # This reference point is from the 0-crack_front_growth_rates_1500ppcf_transformed.csv file for CF_id=0
# reference_x = -1.9932
# reference_y = 363.26

# #         # Find the opposite and adjacent of a triangle formed between reference and given (x,y)
# #         opposite_point = abs(x - reference_x)
# #         adjacent_point = abs(y - reference_y)

# #         # If the line isn't exactly vertical (i.e.: does not have undefined slope)
# #         if adjacent_point != 0:
# #             theta_point = atan(opposite_point / adjacent_point)
            
# #             # If the given point (x,y) is to the right of the reference point, then we got the wrong theta and we need to flip it
# #             if y > reference_y:
# #                 theta_point = pi - theta_point
                
# #             # Formula from the point of reference to the point being tested: x=ay+c
# #             if on_crack_front:
# #                 a = (x - reference_x) / (y - reference_y) # Rise over Run. X and Y are switched in this data
# #                 c = -a * y + x # Solve x=ay+c for c and stick a point in
# #             else:
# a = (actual_x - reference_x) / (actual_y - reference_y) # Rise over Run. X and Y are switched in this data
# c = -a * actual_y + actual_x # Solve x=ay+c for c and stick a point in
# print('a: ', a)
# print('c: ', c)

# #         # Otherwise, the line is vertical, so set theta to pi halves and say we don't know a and c
# #         else:
# #             theta_point = pi / 2
# #             a = None
# #             c = None
            
            
# if a != 0:
#     b = -1 / a
#     d = (1 / a) * actual_y + actual_x
# else:
#     b = None
#     d = None

# # If we found both lines
# if a is not None and b is not None:
#     intersection_y = (d - c) / (a - b) # Remember X and Y are flipped
#     intersection_x = a * intersection_y + c
# # Otherwise, we're missing a line
# else:
#     # If we got the line from the reference to the point (x,y), but not the crack front line
#     if a is not None:
#         if on_crack_front:
#             intersection_y = crack_1y # We know the crack front line is vertical and y is always the same
#         else:
#             intersection_y = y
#         intersection_x = a * intersection_y + c

#     # Otherwise, if we got the crack front line but not the point line
#     elif b is not None:
#         intersection_y = y # The point line is vertical, so its y is always the same
#         intersection_x = b * intersection_y + d

#     # This shouldn't happen cause they are parallel or coincident if it does...If it does I'll figure out how to deal with it
#     else:
#         raise RuntimeError('Both lines are vertical...')

# distance_point = ((y - reference_y)**2 + (x - reference_x)**2)**.5
# distance_intersection = ((intersection_y - reference_y)**2 + (intersection_x - reference_x)**2)**.5

# if on_crack_front:
#     if distance_point >= distance_intersection:
#         in_front_of_crack_surface[(x,y)] = True
# #         print(True)
#     else:
#         not_in_front_of_crack_surface[(x,y)] = False
# #         print(False)
# else:
#     on_line = b * y + d # Find the x coordinate of the perpendicular line
#     print(on_line)
#     if x >= on_line: # If the point we're checking is above or on that line in the x direction
#         print(True)
#     else:
#         print(False)

a:  0.6181006054816107
c:  -226.52442594724988
24.040567513039264
False
